# Design of Experiments for Varying Feeds, Speeds, Depths of Cut

3-level, 3-factor full factorial DoE design

27 total conditions per tool

# Constant Feeds and Speeds, Varying Depth of Cut

1-factor, 5-level DoE design

cutting depth: (5, 10, 15, 20, 25) percent of diameter

5 Repetitions

25 total conditions



Assuming a maximum total data acquisition and inference time of 6 seconds, it should be possible to perform 2 samples for each pass. Use Node Red to ensure exactly 2 samples are captured.

Sample at 32kHz. Use a 32k point FFT to get frequency resolutions of 1Hz. Nyquist Frequency is 16kHz. The PSD from Welch's method will have 16k points. Sampling for 2 seconds, each vibration sample will contain 64k points.


Trigger Sample when feed begins. Flag each with an itemInstanceId for the sample




In [27]:
import numpy as np
import pandas as pd
np.random.seed(123456)

In [28]:
surfaceSpeed = np.array([200,250,300])
cuttingDepth = np.array([5,15,25])
chipLoad = np.array([0.001,0.002,0.003])

toolDiam = 0.625
toothCount = 4

validCount = 10
nCols = 3

maxDepth = 1.8

scalingArray = np.ones((validCount,nCols))
scalingArray[:,0] = 100
scalingArray[:,1] = 20
scalingArray[:,2] = 0.002


shiftingArray = np.ones((validCount,nCols))
shiftingArray[:,0] = 200
shiftingArray[:,1] = 5
shiftingArray[:,2] = 0.001

validDoE = np.random.random((validCount,nCols)) * scalingArray + shiftingArray

In [29]:
DoE = np.zeros((27,3))
columns = ['Surface Speed','Percent Axial Depth','Chip Load','Axial Stepover','Ending Depth']

In [30]:
counter = 0
for i in range(3):
    for j in range(3):
        for k in range(3):
            DoE[counter,:] = np.array([surfaceSpeed[i],cuttingDepth[j],chipLoad[k]])
            counter +=1
np.random.shuffle(DoE)

fullDoE = np.vstack((DoE,validDoE))

In [31]:
axialStep = np.atleast_2d(fullDoE[:,1] * toolDiam / 100).T
matDepth = np.cumsum(axialStep)

In [32]:
while np.any(matDepth > maxDepth):
    matDepth[matDepth > maxDepth] -= matDepth[matDepth > maxDepth][0]
matDepth = np.atleast_2d(matDepth).T

In [33]:
fullDoE = np.hstack((fullDoE,axialStep,matDepth))
fullDoE = np.round(fullDoE,4)

In [34]:
DoEDF = pd.DataFrame(fullDoE,columns=columns)
DoEDF.to_csv('DoE.csv')